In [ ]:
%reset -f

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, date
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from calendar import monthrange
from operator import add, sub
import re
import os

In [ ]:
# Define paths

base = os.path.basename(os.getcwd())
path = os.path.abspath(os.getcwd())

machine = "windows"

if machine == "windows":
    mtu_folder = path[:-len(base)] + "mtu" + "\\"
    mtu_data = mtu_folder + "data" + "\\"
    mtu_file_name = mtu_data + "MTU_"
    mtu_individual = mtu_folder + "individual" + "\\"

In [ ]:
# Select year and month fo analysis

month = 12
year = 2021
days = [str(i).zfill(2) for i in range(1, monthrange(year, month)[1] + 1)]
mtu_dict = dict()

for item in days:
    day_date = date(year,month, int(item))
    new_name = mtu_file_name + day_date.strftime("%b%d_%Y").upper() + ".txt"
    mtu_dict[item] = pd.read_csv(new_name, delimiter = "\t")

In [ ]:
# Individual sensor 
sensors = ["P" + str(i) for i in range(1, 73)]

# Resolution and size of the figure
mydpi = 200
ancho = 1920
ratio = 10/4

# Create a new dataframe for indexing statistical information
rows = ['MEDIA [mV]', 'STD [%]', 'MIN [mV]','MAX [mV]']
df_relacion = pd.DataFrame(columns= rows, index=days)
df_relacion.index.name = 'DÍA'
relacion_dict = dict()
mean_dict = dict()
std_dict = dict()

for sensor in sensors:
    mean = []
    std = []

    for day in days:
        df_relacion[rows[0]][int(day) - 1] = round(mtu_dict[days[int(day) - 1]][sensor].mean(), 2)
        mean.append(mtu_dict[days[int(day) - 1]][sensor].mean())
        if mtu_dict[days[int(day) - 1]][sensor].mean() == 0:
            df_relacion[rows[1]][int(day) - 1] = round(0, 2)
        else:
            df_relacion[rows[1]][int(day) - 1] = round(100*((mtu_dict[days[int(day) - 1]][sensor].std())/(mtu_dict[days[int(day) - 1]][sensor].mean())), 2)
        std.append(mtu_dict[days[int(day) - 1]][sensor].std())
        df_relacion[rows[2]][int(day) - 1] = round(mtu_dict[days[int(day) - 1]][sensor].min(), 2)
        df_relacion[rows[3]][int(day) - 1] = round(mtu_dict[days[int(day) - 1]][sensor].max(), 2)
    
    relacion_dict[sensor] = df_relacion
    mean_dict[sensor] = mean
    std_dict[sensor] = std
    df_relacion.to_excel(mtu_individual + date(year,month, 1).strftime("%b_%Y").upper() + "\\" + sensor + ".xlsx")

    # Here goes the plot
    fig, ax = plt.subplots(1, figsize= ( ancho/mydpi , (ancho/ratio)/mydpi ) , dpi = mydpi)
    plt.rcParams.update({'font.family':'Courier New'})
    mu1 = mean
    sigma1 = std
    indices = list(map(int, days))
    ax.plot(indices, mu1, lw=2, label='mean population 1', color='blue')
    ax.fill_between(indices, list( map(add, mu1, sigma1) ), list( map(sub, mu1, sigma1) ), facecolor='blue', alpha=0.4)
    ax.grid()
    res = list(map(int,re.findall(r'\d+', sensor)))
    month2 = date(year,month, 1).strftime("%b").upper()
    ax.set(xlabel = "Día", ylabel = "Tensión [mV]", title = f"Panel {res[0]} - {month2}", xticks = np.arange(1, int(days[-1]) + 1), xlim = [1, int(days[-1]) + 1])
    plt.savefig(mtu_individual + date(year,month, 1).strftime("%b_%Y").upper() + "\\" + sensor + ".png")


